# Aplicando Transformers en Reconocimiento de Personalidad.

**Investigadores**: <br>
  Dr. Ramón Zatarain Cabada<br>
  Dra. María Lucía Barrón Estrada<br>
  M.C. Víctor Manuel Bátiz Beltrán

# PersonText

En este cuaderno se muestra el uso de Transformers con el corpus PersonText.

**Referencias**:

- Bátiz Beltrán, V. M., Zatarain Cabada, R., & Barrón Estrada, M. L. (2024). Creation of a Corpus in Spanish for the recognition of personality traits. Computación y Sistemas, Volumen 28, No. 3, 2024. ISSN: 2007-9737. Artículo disponible en línea: https://cys.cic.ipn.mx/ojs/index.php/CyS/article/view/4619/3734

# Instalación de bibliotecas

In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinu

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install datasets

#Importación de bibliotecas

In [6]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
import re
import emoji
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

ModuleNotFoundError: No module named 'datasets'

#Procedimiento

## 1.Carga del conjunto de Datos

## Descarga del Corpus

In [1]:
def corpus_download(path, url):
  !wget --no-check-certificate \
     {url} \
     -O {path}

In [2]:
corpus_download("persontext.csv","https://person-app-itc.web.app/corpus/corpus_persontext_shuffle.csv")
corpus_download("persontextV2.csv","https://person-app-itc.web.app/corpus/corpus_persontext_shufflev2.csv")
corpus_download("mypersonality.csv","https://person-app-itc.web.app/corpus/corpus_mypersonality_pt_esp.csv")

--2024-10-18 16:59:48--  https://person-app-itc.web.app/corpus/corpus_persontext_shuffle.csv
Resolving person-app-itc.web.app (person-app-itc.web.app)... 199.36.158.100, 2620:0:890::100
Connecting to person-app-itc.web.app (person-app-itc.web.app)|199.36.158.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 159288 (156K) [text/csv]
Saving to: ‘persontext.csv’

persontext.csv      100%[===================>] 155.55K  --.-KB/s    in 0.02s   

2024-10-18 16:59:49 (7.58 MB/s) - ‘persontext.csv’ saved [159288/159288]



In [8]:
#corpusPath = "mypersonality.csv"
#corpusPath = "persontext.csv"
#corpus = pd.read_csv(corpusPath)

In [9]:
#corpus.tail()

,uid,texto,fecha_prueba,apertura,presenta_apertura,responsabilidad,presenta_responsabilidad,sociabilidad,presenta_sociabilidad,amabilidad,presenta_amabilidad,neuroticismo,presenta_neuroticismo,fechaNacimiento,sexo,edad_calculada,Experimento
208,DOIxrvGJKHexpjsBAPh4zf5eXyQ2,Bueno chavos vamos a ponernos pilas con el tr...,01/11/2021,0.5,No,0.8,Sí,0.4,No,0.8,Sí,0.3,No,06/02/1999,Masculino,22,Exp 3-1
209,B6AJrl8ECWe2rIUCiO5TuYxeLwl1,Bueno pues considero yo que para mi persona e...,04/03/2021,0.7,Sí,0.8,Sí,0.7,Sí,0.5,No,0.9,Sí,02/12/1995,Masculino,25,Exp 1-2
210,iXWwNMyEN5OBn4F6nMQnoqxNoki2,Hola el video de hoy se tratará más que nada ...,04/03/2021,0.8,Sí,0.8,Sí,0.3,No,0.8,Sí,0.9,Sí,23/07/1994,Masculino,26,Exp 1-3
211,n3a5RSlooCZYAuoefM4gCirmh9I2,Son cinco las características que debe tener ...,04/03/2021,0.5,No,0.5,No,0.6,No,0.6,No,0.7,Sí,24/10/1995,Masculino,25,Exp 1-3
212,m8zpZe9PXmQ9tRLjB7pVSCZwoYv2,Bueno pues un tema que me interesa a mí son l...,26/03/2021,0.9,Sí,0.7,Sí,0.2,No,0.7,Sí,0.6,No,10/07/1997,Masculino,23,Exp 1-1


## Selección del Corpus a utilizar.

In [11]:
corpus_dict = {
    'PersonText' : 'persontext.csv',
    'PersonTextV2' : 'persontextV2.csv',
    'MyPersonality' : 'mypersonality.csv'
}

selectedCorpusName = 'MyPersonality' #@param ['PersonText','PersonTextV2','MyPersonality']
corpus = pd.read_csv(corpus_dict[selectedCorpusName])


## Exploración del Corpus

In [12]:
corpus.tail()

,Unnamed: 0,uid,texto,fecha_prueba,apertura,presenta_apertura,responsabilidad,presenta_responsabilidad,sociabilidad,presenta_sociabilidad,amabilidad,presenta_amabilidad,neuroticismo,presenta_neuroticismo,fechaNacimiento,sexo,edad_calculada,Experimento
2324,2324,d2ee2871aa02698e66fdc72b2218cdd8,"¡Vaya, finalmente configuré mi computadora en ...",NaN,4.65,y,4.0,y,2.75,n,2.50,n,3.25,y,NaN,NaN,NaN,NaN
2325,2325,ea890531d3e0547166efde52d843099e,Solo quería agradecer a todos por todo el apoy...,NaN,5.00,y,3.5,y,4.75,y,3.75,y,4.25,y,NaN,NaN,NaN,NaN
2326,2326,ea890531d3e0547166efde52d843099e,¿Sabes cuando trabajas tu @$$ durante mucho ti...,NaN,5.00,y,3.5,y,4.75,y,3.75,y,4.25,y,NaN,NaN,NaN,NaN
2327,2327,fbe5aa478508d1dc931427ade5d9e1b4,Oh bien. Solo hay dos cosas que hacer cuando t...,NaN,4.00,y,4.0,y,3.25,n,2.25,n,2.75,n,NaN,NaN,NaN,NaN
2328,2328,a286b7286b1247d4a7851709e9f31e1e,¡Anoche fue increíble! No solo vi la ÚLTIMA pr...,NaN,4.00,y,3.5,y,4.25,y,3.25,n,3.00,y,NaN,NaN,NaN,NaN


## 2. Limpieza de los Datos

Dejamos solo las columnas con los datos que vamos a trabajar.

In [13]:
#                    0                     1                       2                      3                       4
#rasgos = ['presenta_apertura','presenta_responsabilidad','presenta_sociabilidad','presenta_amabilidad','presenta_neuroticismo']

#rasgos = ['presenta_apertura', 'presenta_neuroticismo']

rasgos = ['presenta_apertura']

corpus_limpio = corpus[['texto',*rasgos]]


Visualizamos el Corpus resultante.

In [14]:
corpus_limpio.tail()

,texto,presenta_apertura
2324,"¡Vaya, finalmente configuré mi computadora en ...",y
2325,Solo quería agradecer a todos por todo el apoy...,y
2326,¿Sabes cuando trabajas tu @$$ durante mucho ti...,y
2327,Oh bien. Solo hay dos cosas que hacer cuando t...,y
2328,¡Anoche fue increíble! No solo vi la ÚLTIMA pr...,y


Reemplazamos las etiquetas "Sí" y "No" por "1" y "0" respectivamente.

In [15]:
replacement_mapping = {'Sí': 1, 'No': 0, 'y': 1, 'n':0}
corpus_limpio[rasgos] = corpus_limpio[rasgos].replace(replacement_mapping)

<ipython-input-15-29b9d61f2b29>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  corpus_limpio[rasgos] = corpus_limpio[rasgos].replace(replacement_mapping)
<ipython-input-15-29b9d61f2b29>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus_limpio[rasgos] = corpus_limpio[rasgos].replace(replacement_mapping)


In [16]:
corpus_limpio.tail()

,texto,presenta_apertura
2324,"¡Vaya, finalmente configuré mi computadora en ...",1
2325,Solo quería agradecer a todos por todo el apoy...,1
2326,¿Sabes cuando trabajas tu @$$ durante mucho ti...,1
2327,Oh bien. Solo hay dos cosas que hacer cuando t...,1
2328,¡Anoche fue increíble! No solo vi la ÚLTIMA pr...,1


##3. Inicialización de Modelo y utilidades

Selección de Modelos



In [20]:
modelsNames = {
    'bert-base-multilingual-cased' : 'BERT_b_C_mñ',
    'bert-base-multilingual-uncased': 'BERT_b_U_ml',
    'distilbert-base-uncased' : 'distilBERT_b_U',
    'roberta-base' : 'roBERTa_b',
    'dccuchile/bert-base-spanish-wwm-cased' : 'BETO_b_C',
    'xlnet-base-cased' : 'xlent_b_C'
}

pretrained_Model = "bert-base-multilingual-cased" #@param ['bert-base-multilingual-cased', 'bert-base-multilingual-uncased','distilbert-base-uncased','roberta-base','dccuchile/bert-base-spanish-wwm-cased','xlnet-base-cased']

Definimos la conversión de etiquetas.

In [18]:
id2label = {0: "No", 1: "Si"}
label2id = {"No": 0, "Si": 1}

Definimos los sub conjuntos de 'Train', 'Test', y 'Validation'.

In [19]:
# Define the proportion of data you want for each set (train, test, validation)
train_ratio = 0.70  # 70% for training
test_ratio = 0.15  # 15% for testing
validation_ratio = 0.15  # 15% for validation

# Calculate the test_ratio and validation_ratio in terms of the remaining data
remaining_ratio = test_ratio + validation_ratio
test_ratio = test_ratio / remaining_ratio
validation_ratio = validation_ratio / remaining_ratio

Asignamos el Modelo a usar, en este caso: BERT-Multilingual-Cased

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_Model)
model = AutoModelForSequenceClassification.from_pretrained(
   pretrained_Model, num_labels=2, id2label=id2label, label2id=label2id
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenizer_preprocess_function(examples):
    return tokenizer(examples["texto"], truncation=True)

In [ ]:
#accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir=f"model_saves",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit = 1,
    metric_for_best_model = 'eval_recall'
    #metric_for_best_model = 'eval_accuracy'
    #push_to_hub=True,
)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Realizamos una iteración sobre cada rasgos para entrenar un modelo por cada rasgo.

In [ ]:
#rasgo = 'presenta_apertura'
#rasgo = rasgo.split('_')[1].capitalize()
#print(best_model_save_dir)

In [ ]:
results = {}
results['model'] = []
results['trait'] = []
results['train_runtime'] = []

for rasgo in rasgos:
  nombre_rasgo = rasgo.split('_')[1].capitalize()
  print(f'Entrenando para rasgo: {nombre_rasgo}')
  rasgo_corpus = corpus_limpio[['texto', rasgo]]
  rasgo_corpus = rasgo_corpus.rename(columns={rasgo:'labels'})
  training_args.output_dir = f"{pretrained_Model}_{nombre_rasgo}_checkpoints"

  rasgo_corpus_train, rasgo_corpus_remaining = train_test_split(rasgo_corpus, test_size=remaining_ratio, random_state=42)
  rasgo_corpus_test, rasgo_corpus_validation = train_test_split(rasgo_corpus_remaining, test_size=validation_ratio, random_state=42)

  # Elimina el índice de los DataFrames
  rasgo_corpus_train = rasgo_corpus_train.reset_index(drop=True)
  rasgo_corpus_test = rasgo_corpus_test.reset_index(drop=True)
  rasgo_corpus_validation = rasgo_corpus_validation.reset_index(drop=True)

  # Convierte los DataFrames a datasets
  dataset_train = Dataset.from_pandas(rasgo_corpus_train)
  dataset_test = Dataset.from_pandas(rasgo_corpus_test)
  dataset_validation = Dataset.from_pandas(rasgo_corpus_validation)

  # Crea el DatasetDict
  dataset_dict = DatasetDict({
      'train': dataset_train,
      'test': dataset_test,
      'validation': dataset_validation
  })

  tokenized_data = dataset_dict.map(tokenizer_preprocess_function, batched=True)
  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
  )

  train_data = trainer.train()
  eval_metrics = trainer.evaluate()

  for metric in eval_metrics:
    if metric not in results:
      results[metric] = []
    results[metric].append(eval_metrics[metric])

  results['trait'].append(rasgo.split('_')[-1])
  results['train_runtime'].append(train_data.metrics['train_runtime'])
  results['model'].append(pretrained_Model)


  #best_model_save_dir = f'/content/drive/MyDrive/ModelosPersonalidad/Modelo_{nombre_rasgo}'
  best_model_save_dir = f'/ModelosPersonalidad/Modelo_{nombre_rasgo}'

  trainer.save_model(best_model_save_dir)

  print(results)



Entrenando para rasgo: Apertura


Map:   0%|          | 0/1630 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


...

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall
1,No log,0.666929,0.616046,0.469680,0.616046
2,No log,0.667222,0.616046,0.469680,0.616046
3,No log,0.654428,0.630372,0.534098,0.630372
4,No log,0.690910,0.638968,0.613558,0.638968
5,0.626800,0.835051,0.593123,0.586650,0.593123
6,0.626800,0.931671,0.584527,0.579213,0.584527
7,0.626800,0.986264,0.630372,0.586001,0.630372
8,0.626800,1.071553,0.627507,0.585803,0.627507
9,0.626800,1.168801,0.564470,0.560174,0.564470
10,0.349000,1.183525,0.578797,0.566548,0.578797


{'model': ['roberta-base', 'roberta-base', 'roberta-base', 'roberta-base', 'roberta-base'], 'trait': ['apertura', 'responsabilidad', 'sociabilidad', 'amabilidad', 'neuroticismo'], 'train_runtime': [884.527, 872.6052, 885.4695, 890.8155, 869.395], 'eval_loss': [0.5560699701309204, 0.6941128969192505, 1.518521785736084, 0.8317784667015076, 0.690909743309021], 'eval_accuracy': [0.7507163323782235, 0.5587392550143266, 0.5931232091690545, 0.6189111747851003, 0.6389684813753582], 'eval_f1': [0.6438221901246958, 0.42850441036013265, 0.5918620565145479, 0.620064991788507, 0.6135579637769412], 'eval_recall': [0.7507163323782235, 0.5587392550143266, 0.5931232091690545, 0.6189111747851003, 0.6389684813753582], 'eval_runtime': [4.2092, 4.222, 4.2625, 4.2668, 4.2548], 'eval_samples_per_second': [82.913, 82.662, 81.877, 81.795, 82.025], 'eval_steps_per_second': [5.227, 5.211, 5.161, 5.156, 5.171], 'epoch': [10.0, 10.0, 10.0, 10.0, 10.0]}


In [ ]:
df = pd.DataFrame(results)

In [ ]:
df

,model,trait,train_runtime,eval_loss,eval_accuracy,eval_f1,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
0,roberta-base,apertura,884.5270,0.556070,0.750716,0.643822,0.750716,4.2092,82.913,5.227,10.0
1,roberta-base,responsabilidad,872.6052,0.694113,0.558739,0.428504,0.558739,4.2220,82.662,5.211,10.0
2,roberta-base,sociabilidad,885.4695,1.518522,0.593123,0.591862,0.593123,4.2625,81.877,5.161,10.0
3,roberta-base,amabilidad,890.8155,0.831778,0.618911,0.620065,0.618911,4.2668,81.795,5.156,10.0
4,roberta-base,neuroticismo,869.3950,0.690910,0.638968,0.613558,0.638968,4.2548,82.025,5.171,10.0


In [ ]:
df.to_csv(f'{selectedCorpusName}_{modelsNames[pretrained_Model]}_results.csv', index= False)